In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [4]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv',low_memory=False)

In [7]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [12]:
#df['brand_cat'] = df['brand'].factorize()[0]
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0] #zmiana liter brandu na małą. Normalizacja danych jest fajna i usprawnia! Bo ktoś mógł wpisać ten sam brend z małej i dużej litery
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [13]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [16]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [17]:
test = {'key': 'value'}
test['key']

'value'

In [18]:
str(test)

"{'key': 'value'}"

In [20]:
#Trzeba ten string przywrócić do postaci słownnika
str_dict='[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features1(x):
  if str(x)=='nan': return []
  return literal_eval(x.replace('\\"', '"') )

def parse_features(x):
  output_dict= {}
  if str(x)=='nan': return output_dict
  features = literal_eval(x.replace('\\"', '"') )
  for item in features: #{'key': 'Gender', 'value': ['Men']}
    key = item['key']
    value = item['value'][0].lower().strip() #strip usuwa spacje
    output_dict[key] = value
    
  return output_dict

    
df['features_parsed'] = df['features'].map(parse_features)

In [25]:
df['features_parsed'].head().values

array([list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in pounds)', 'value': ['0.45']}, {'key': 'Condition', 'value': ['New']}, {'key': 'Brand', 'value': ['SERVUS BY HONEYWELL']}, {'key': 'manufacturer_part_number', 'value': ['ZSR101BLMLG']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in

In [30]:
df['features_parsed'].head().values

array([{'Gender': 'men', 'Shoe Size': 'm', 'Shoe Category': "men's shoes", 'Color': 'multicolor', 'Manufacturer Part Number': '8190-w-navy-7.5', 'Brand': 'josmo'},
       {'Gender': 'men', 'Shoe Size': 'm', 'Shoe Category': "men's shoes", 'Color': 'multicolor', 'Manufacturer Part Number': '8190-w-navy-7.5', 'Brand': 'josmo'},
       {'Gender': 'men', 'Color': 'black', 'Shipping Weight (in pounds)': '0.45', 'Condition': 'new', 'Brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'Gender': 'men', 'Color': 'black', 'Shipping Weight (in pounds)': '0.45', 'Condition': 'new', 'Brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'Gender': 'men', 'Color': 'black', 'Shipping Weight (in pounds)': '0.45', 'Condition': 'new', 'Brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [31]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()) )
len(keys)

490

In [34]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
df.columns
pusta = pd.DataFrame(index=df.columns)
pusta.to_csv('/content/kolumny.csv')

In [49]:
df[ df['feat_Athlete'].isnull() ].shape

(18272, 540)

In [50]:
df.shape[0]

18280

In [52]:
df[False == df['feat_Athlete'].isnull() ].shape[0] / df.shape[0] * 100 #cecha występuje w 0.04% przypadków i nie chcemy jej brać pod uwagę

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [58]:
{k: v for k,v in keys_stat.items() if v > 30 }

{'Brand': 48.62691466083151,
 'Color': 47.784463894967175,
 'Gender': 50.10393873085339,
 'Manufacturer Part Number': 36.252735229759296,
 'Material': 34.18490153172867}

In [0]:
df['feat_brand_cat'] = df['feat_Brand'].factorize()[0]
df['feat_color_cat'] = df['feat_Color'].factorize()[0]
df['feat_gender_cat'] = df['feat_Gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_Manufacturer Part Number'].factorize()[0]
df['feat_material_cat'] = df['feat_Material'].factorize()[0]

df['feat_sport_cat'] = df['feat_Sport'].factorize()[0]
df['feat_style_cat'] = df['feat_Style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [0]:
run_model(['brand_cat'])

In [61]:
df[ df.brand != df.feat_Brand].shape

(18228, 545)

In [62]:
df[ df.brand == df.feat_Brand][ ['brand', 'feat_Brand']].head() 

,brand,feat_Brand
2163,adidas,adidas
2417,adidas,adidas
3021,adidas,adidas
3054,adidas,adidas
3629,totes,totes


In [64]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand != df.feat_Brand][ ['brand', 'feat_Brand']].head(10) 

,brand,feat_Brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN
32,toms,NaN
33,toms,NaN
34,venum,NaN
35,azzuro,NaN
36,azzuro,NaN


In [83]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_Multi Pack Indicator',
 'feat_Location - City/State',
 'feat_Location - Country',
 'feat_Recommended Location',
 'feat_Certifications and Listings',
 'feat_Fabrication',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_Series Name_cat',
 'feat_Fuel Type_cat',
 'feat_Lens Color_cat',
 'feat_FIT_cat',
 'feat_Rx-able_cat',
 'feat_Is Orthopedic_cat',
 'feat_Polarized Lenses_cat',
 'feat_Is Wheeled_cat',
 'feat_Instrument_cat',
 'feat_Jewelry Setting_cat',
 'feat_Item weight_cat',
 'feat_Face Shape_cat',
 'feat_Cushioning_cat',
 'feat_Item color_cat',
 'feat_Year Made_cat',
 'feat_Chest_cat',
 'feat_Best Sellers Rank_cat',
 'feat_Shirt Size_cat',
 'feat_Metal Color_cat',
 'feat_Material Composition_cat',
 'feat_Movement_cat',
 'feat_Clothing Size Type_cat',
 'feat_Polarized?_cat',
 'feat_Contained Battery Type_cat',
 'feat_SKU Number_cat',
 'f

In [65]:
feats = ['']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.37239326123891, 4.225197434001087)

In [0]:
feats = ['brand_cat','feat_Metal Type_cat','feat_Shape_cat','feat_Country of Origin - Assembly_cat','feat_Fabric Content_cat','feat_Case Diameter_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_sport_cat'] #'feat_manufacturer part number_cat' 'feat_color_cat', #usuwamy nieistotną cechę
#feats += feats_cat
# feats += ['feat_EAN_cat',
#  'feat_Type_cat',
#  'feat_Water-Resistant_cat',
#  'feat_Age Group_cat',
#  'feat_Work Shoes_cat',
#  'feat_Waterproof_cat',
#  'feat_Country//Region of Manufacture_cat',
#  'feat_Lens_cat',
#  'feat_Szie_cat',
#  'feat_Part Number_cat',
#  'feat_Part Type_cat',
#  'feat_Dial Color_cat',
#  'feat_NAVY BLUE_cat',
#  'feat_Foot Arch_cat',
#  'feat_To Fit_cat',
#  'feat_Capacity_cat',
#  'feat_Eye Size:_cat',
#  'feat_Frame Style_cat',
#  'feat_Mechanic_cat',
#  'feat_Genre_cat',
#  'feat_Handbag Style_cat',
#  'feat_Protection_cat',
#  'feat_Resolution_cat',
#  'feat_Frame Depth_cat',
#  'feat_Number of Batteries Included_cat',
#  'feat_Clothing Type_cat',
#  'feat_Weight_cat',
#  'feat_Frame Color_cat',
#  'feat_Is Waterproof_cat',
#  'feat_Sports League_cat',
#  'feat_Fine or Fashion_cat',
#  'feat_Walmart No._cat',
#  'feat_Band Material_cat',
#  'feat_Age End_cat',
#  'feat_Gold Karat_cat',
#  'feat_Item Package Quantity_cat',
#  'feat_Motion Control_cat',
#  'feat_Frame Shape_cat',
#  'feat_Product Type_cat',
#  'feat_Condition_cat',
#  'feat_amazonBestSellersRank_cat',
#  'feat_Temple Length_cat',
#  'feat_Sleeve Length_cat',
#  'feat_Animal Type_cat',
#  'feat_Country of Origin - Assembly_cat',
#  'feat_Lens Tint_cat',
#  'feat_Black_cat',
#  'feat_Attachment_cat',
#  'feat_Number of Heat Settings_cat',
#  'feat_Retail Price_cat',
#  'feat_Impact Resistant_cat',
#  'feat_Stock No._cat',
#  'feat_Material Type_cat',
#  'feat_Fabric Content_cat',
#  'feat_Case Diameter_cat',
#  'feat_Closure style_cat',
#  'feat_Technician_cat',
#  'feat_Enhanced Visibility (Reflective)_cat',
#  'feat_Labor_cat',
#  'feat_Width_cat',
#  'feat_Материал_cat',
#  'feat_Frame material:_cat',
#  'feat_Garment Care_cat',
#  'feat_Power Reserve_cat',
#  'feat_Inseam_cat',
#  'feat_Wind Resistant_cat',
#   'feat_Band Length_cat',
#  'feat_Crystal_cat',
#  'feat_Frame Type_cat',
#  'feat_Feature_cat',
#  'feat_Fabric Care_cat',
#  'feat_Pocket_cat',
#  'feat_Item Spec Attr - Style (eBay)_cat',
#  'feat_Hunting_cat',
#  'feat_Manufacturer Number_cat',
#  'feat_Vehicle Type_cat',
#  'feat_Comfort Technology_cat',
#  'feat_UV Protection_cat',
#  'feat_Label_cat',
#  'feat_Auto_cat',
#  'feat_Country of Origin Components:_cat',
#  'feat_Country/Region of Manufacture_cat',
#  'feat_Is Polarized_cat',
#  'feat_ISBN_cat',
#  'feat_Military Style_cat',
#  'feat_Casing_cat',
#  'feat_Country/ Organization_cat',
#  'feat_Antiscratch lens coating_cat',
#  'feat_Diamond Color_cat',
#  'feat_Hood_cat',
#  'feat_Country of Origin_cat',
#  'feat_Leg_cat',
#  'feat_ADIDAS_cat',
#  'feat_Clothing Product Type_cat',
#  'feat_Watch Case Shape_cat',
#  'feat_Expandable_cat',
#  'feat_Applicable_cat',
#  'feat_Overall Material_cat',
#  'feat_Main Color_cat',
#  'feat_Target Audience_cat',
#  'feat_Machine Washable_cat',
#  'feat_Fit_cat',
#  'feat_Watch Power Source_cat',
#  'feat_Model Number_cat',
#  'feat_Article_cat',
#  'feat_Heel Height_cat']
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [104]:
X = df[ feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)
print('Result: ', result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Result:  (-58.39407009657966, 4.850114618603302)


Weight,Feature
0.2434 ± 0.0110,brand_cat
0.0564 ± 0.0013,feat_brand_cat
0.0169 ± 0.0053,feat_style_cat
0.0165 ± 0.0016,feat_Case Diameter_cat
0.0100 ± 0.0015,feat_Metal Type_cat
0.0089 ± 0.0007,feat_Fabric Content_cat
0.0043 ± 0.0003,feat_Country of Origin - Assembly_cat
0.0014 ± 0.0001,feat_Shape_cat
0.0013 ± 0.0001,feat_material_cat
0.0001 ± 0.0000,feat_sport_cat


In [73]:
df['brand'].value_counts(normalize=True)

nike             0.097210
puma             0.033315
ralph lauren     0.028775
vans             0.021116
new balance      0.020295
                   ...   
tailian          0.000055
onetigris        0.000055
akadema          0.000055
david spencer    0.000055
dr. zen          0.000055
Name: brand, Length: 1732, dtype: float64

In [75]:
df [df['brand'] == 'nike'].features_parsed.head().values

array([{'Sport': 'soccer', 'Main Color': 'orange', 'Type': 'cleats', 'Condition': 'new without box'},
       {'Sport': 'soccer', 'Main Color': 'orange', 'Type': 'cleats', 'Condition': 'new without box'},
       {'Sport': 'soccer', 'Main Color': 'orange', 'Type': 'cleats', 'Condition': 'new without box'},
       {'Style': 'athletic sneakers', 'Condition': 'new with box'}, {}],
      dtype=object)

In [0]:
df['feat_Metal Type_cat']

In [101]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [124]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	dayy5.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [0]:
!git add matrix_one/day3b.ipynb

In [114]:
!git commit -m "feature engineering cech obuwia i ważność cech"

[master 72cd8b0] feature engineering cech obuwia i ważność cech
 2 files changed, 542 insertions(+), 1 deletion(-)
 create mode 100644 kolumny.csv


In [0]:
!git config --global user.email "interes@go2.pl"
!git config --global user.name "Rafall"

In [122]:
!git push -u origin master

Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 4.80 KiB | 819.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/rafallysik/dw_matrix.git
   5ef1d09..72cd8b0  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.


In [120]:
!git remote -v

origin	https://a908fbad6ebae1b8c9abdd093be779819e7465da@github.com/rafallysik/dw_matrix.git (fetch)
origin	https://a908fbad6ebae1b8c9abdd093be779819e7465da@github.com/rafallysik/dw_matrix.git (push)


In [0]:
!git remote set-url origin https://mojklucz@github.com/rafallysik/dw_matrix.git